In [106]:
import pandas as pd
import config as c
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

In [107]:
data = pd.read_csv('data/runs/results.csv', index_col='run')

In [108]:
data = data[data['status']!='error']
data = data[data['r_2_mean'].notnull()]

In [109]:
data['time'] = pd.to_timedelta(data['time'])
data['started'] = pd.to_datetime(data['started'])

In [110]:
data['seconds'] = data['time'].apply(lambda x:x.seconds)

In [111]:
data['datasets'] = data['gdsc']+data['ctrp']+data['ccle']

In [112]:
data = data[data['datasets']>0]

In [113]:
ndata = data[data['started'] > pd.to_datetime('2020-04-15 18:00:00')]

In [114]:
data[(data['datasets']>1)&(data['feda'] == True)].describe()

,time,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds
count,268,268.0,268.000000,265.000000,0.0,268.000000,268.0,268.000000,268.000000,268.000000
mean,0 days 01:23:42.955223,4.0,20.608209,0.099655,NaN,0.010485,0.0,-0.027281,0.033133,5022.955224
std,0 days 01:56:36.946512,0.0,4.575567,0.062088,NaN,0.006008,0.0,0.033966,0.064889,6996.946513
min,0 days 00:01:02,4.0,3.000000,0.000000,NaN,0.010000,0.0,-0.192936,0.001156,62.000000
25%,0 days 00:26:00.750000,4.0,20.000000,0.047106,NaN,0.010000,0.0,-0.043439,0.007424,1560.750000
50%,0 days 00:54:12,4.0,20.000000,0.097392,NaN,0.010000,0.0,-0.009616,0.013535,3252.000000
75%,0 days 01:39:40.250000,4.0,20.000000,0.152094,NaN,0.010000,0.0,-0.004624,0.035752,5980.250000
max,0 days 17:16:36,4.0,50.000000,0.245415,NaN,0.100000,0.0,-0.001289,0.721564,62196.000000


In [115]:
data.describe()

,time,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds
count,1139,1140.0,1140.000000,1131.000000,0.0,1140.000000,1140.0,1140.000000,1139.000000,1139.000000
mean,0 days 00:47:56.871817,4.0,21.611404,0.101838,NaN,0.010272,0.0,0.293297,0.267094,2876.871817
std,0 days 01:14:43.190530,0.0,6.928637,0.060228,NaN,0.004761,0.0,0.579034,1.899436,4483.190531
min,0 days 00:00:35,4.0,1.000000,0.000000,NaN,0.010000,0.0,-12.995511,0.000012,35.000000
25%,0 days 00:10:29.500000,4.0,20.000000,0.053639,NaN,0.010000,0.0,-0.004962,0.013836,629.500000
50%,0 days 00:28:04,4.0,20.000000,0.097598,NaN,0.010000,0.0,0.148576,0.109957,1684.000000
75%,0 days 00:54:09,4.0,20.000000,0.152284,NaN,0.010000,0.0,0.595782,0.176025,3249.000000
max,0 days 17:16:36,4.0,50.000000,0.248831,NaN,0.100000,0.0,0.999979,58.519798,62196.000000


## FEDA t-test

In [116]:
data[data['fs']=='Lasso'][['model', 'feda','r_2_mean']]

,model,feda,r_2_mean
run,,,
66a296a7,ElasticNet,True,-0.005487
51241a74,ElasticNet,True,-0.002197
bfc2ac8f,KNeighborsRegressor,False,0.094956
6e9ab9d6,RandomForestRegressor,False,0.358532
dab89ab7,ElasticNet,True,-0.007901
...,...,...,...
ec5e4047,DecisionTreeRegressor,False,0.999100
81ec90e7,DecisionTreeRegressor,False,0.999535
dde14bbd,DecisionTreeRegressor,False,0.999529


In [117]:
from scipy import stats
stats.ttest_ind(data[data['metric']=='AUC_IC50']['r_2_mean'].values, data[data['metric']!='AUC_IC50']['r_2_mean'].values)

Ttest_indResult(statistic=-0.2292471474940539, pvalue=0.8187180228623517)

In [118]:
stats.ttest_ind(data[data['feda']==True]['r_2_mean'].values, data[data['feda']==False]['r_2_mean'].values, equal_var=False)

Ttest_indResult(statistic=-9.573756472531846, pvalue=7.63829048691804e-21)

## Model composition

In [119]:
models = data.groupby('model').mean()
models['Number of runs'] = [data[data['model']==i].shape[0] for i in models.index]
models

,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds,Number of runs
model,,,,,,,,,,
DecisionTreeRegressor,4.0,19.913242,0.098092,NaN,0.010000,0.0,0.499668,0.103612,2519.068493,219
ElasticNet,4.0,20.000000,0.097647,NaN,0.010000,0.0,0.145365,0.120638,1901.148889,450
KNeighborsRegressor,4.0,20.000000,0.088223,NaN,0.010000,0.0,0.163838,0.089808,10209.366667,60
RandomForestRegressor,4.0,19.914573,0.099561,NaN,0.010000,0.0,0.607753,0.043194,4288.778894,199
SVR,4.0,28.834906,0.121425,NaN,0.011462,0.0,0.135586,1.006423,1912.488152,212


In [120]:
nmodels = data[data['started'] > pd.to_datetime('2020-04-15 18:00:00')].groupby('model').mean()
nmodels['Number of runs'] = [data[(data['model']==i) & (data['started'] > pd.to_datetime('2020-04-15 18:00:00'))].shape[0] for i in nmodels.index]
nmodels

,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds,Number of runs
model,,,,,,,,,,
DecisionTreeRegressor,4.0,20.0,0.091618,NaN,0.01,0.0,-0.056516,0.284986,3975.285714,49
ElasticNet,4.0,20.0,0.092367,NaN,0.01,0.0,0.110539,0.112015,2467.656250,96
KNeighborsRegressor,4.0,20.0,0.095194,NaN,0.01,0.0,0.066489,0.099449,13622.216216,37
RandomForestRegressor,4.0,20.0,0.082255,NaN,0.01,0.0,0.120448,0.098267,4300.627451,51
SVR,4.0,20.0,0.066008,NaN,0.01,0.0,0.023019,0.265870,1862.000000,22


## Individual drug analysis

In [121]:
svr = data[(data['model'] == 'SVR') & (data['r_2_mean']>0)]
svr[svr['r_2_mean']> 0.6][['feda', 'n', 'fs','r_2_mean']].sort_values('r_2_mean', ascending=False)

,feda,n,fs,r_2_mean
run,,,,
c84c194d,False,0.010000,ElasticNet,0.934915
8071f94a,False,0.094795,DecisionTreeRegressor,0.922619
400d39b7,False,0.098271,DecisionTreeRegressor,0.922580
d2703cd1,False,0.147682,mutual_info_regression,0.919401
dfde6ba7,False,0.072084,mutual_info_regression,0.917935
c889fa24,False,0.087590,f_regression,0.916772
b1cea166,False,0.003354,mutual_info_regression,0.901983
e32273a7,False,0.144099,ElasticNet,0.897122
9cabe208,False,0.057282,RandomForestRegressor,0.887763


In [122]:
en = data[(data['model'] == 'ElasticNet') & (data['r_2_mean']>0)]
en[['feda', 'n', 'fs','r_2_mean']].sort_values('r_2_mean', ascending=False)

,feda,n,fs,r_2_mean
run,,,,
150587db,False,0.100081,mutual_info_regression,0.541867
140511c0,False,0.006392,DecisionTreeRegressor,0.502989
61f79f6a,False,0.146809,DecisionTreeRegressor,0.499752
a0fd39fd,False,0.161861,DecisionTreeRegressor,0.479983
d122f81c,True,0.002680,DecisionTreeRegressor,0.445192
...,...,...,...,...
ae6aad99,True,0.083612,f_regression,0.030006
30aec5a1,True,0.172041,mutual_info_regression,0.029708
c7025afd,False,0.025666,RandomForestRegressor,0.021164


In [123]:
dt = data[(data['model'] == 'DecisionTreeRegressor') & (data['r_2_mean']>0)]
dt[['feda', 'n', 'fs','r_2_mean']].sort_values('r_2_mean', ascending=False)

,feda,n,fs,r_2_mean
run,,,,
bb21f1a1,False,0.178835,Lasso,0.999979
ffb505bc,False,0.029654,Lasso,0.999978
8ee1461e,False,0.000266,Lasso,0.999976
62069de6,False,0.132054,Lasso,0.999972
43650dda,False,0.073360,Lasso,0.999966
...,...,...,...,...
25e24ad2,False,0.117603,RandomForestRegressor,0.110713
b54c0127,False,0.109115,ElasticNet,0.100940
1ea7b30d,False,0.018304,ElasticNet,0.056617


In [124]:
def append(data, run):
    result = pd.DataFrame()
    for a in run:
        f = subprocess.run("guild select " +a, shell=True, capture_output=True)
        d = pd.read_csv('venv/.guild/runs/'+f.stdout.decode('utf-8')[:-1]+'/scores.csv')
        d['run'] = [a for i in range(0, d.shape[0])]
        d = d.set_index('run')
        result = pd.concat([result, d])
    return result

In [125]:
%%time
k = append(data, data.index)

CPU times: user 29.5 s, sys: 8.88 s, total: 38.4 s
Wall time: 1h 43min 45s


In [127]:
k.to_csv('data/runs/drug_data.csv')

In [128]:
pd.read_csv('data/runs/drug_data.csv')

/Users/admin/Desktop/Thesis/Code/phc-data-pipeline/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (32,33,34,36,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,run,Unnamed: 0,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,alpha,copy_X,fit_intercept,l1_ratio,...,p,weights,C,cache_size,coef0,degree,epsilon,gamma,kernel,shrinking
0,ffd4d4a7,17-AAG,0.387154,0.181439,0.050514,0.154822,1.0,True,True,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ffd4d4a7,Crizotinib,0.263440,0.261585,0.091475,0.270581,0.5,True,True,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ffd4d4a7,Erlotinib,0.274212,0.144041,0.037329,0.113591,5.0,True,True,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ffd4d4a7,Lapatinib,0.270125,0.153267,0.041427,0.112792,5.0,True,True,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ffd4d4a7,Nilotinib,0.158333,0.135443,0.040566,0.080521,5.0,True,True,0.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21558,bee8f3c7,L-685458,0.692639,0.077941,0.007216,0.084736,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21559,bee8f3c7,Lapatinib,0.892844,0.063590,0.005634,0.062192,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21560,bee8f3c7,TAE684,0.909815,0.064560,0.006383,0.058898,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21561,bee8f3c7,Saracatinib,0.862349,0.069663,0.007291,0.061629,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
lo = k.join(ndata, lsuffix='n')

In [140]:
lo[lo['Unnamed: 0']=='17-AAG']

,Unnamed: 0,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,alpha,copy_X,fit_intercept,l1_ratio,max_iter,...,metric,model,n,test,threshold,step,r_2_mean,r_2_std,seconds,datasets
run,,,,,,,,,,,,,,,,,,,,,
00285b88,17-AAG,0.428393,0.185934,5.240038e-02,0.165394,5.0,True,True,0.00,5000.0,...,AUC_IC50,ElasticNet,0.131306,NaN,0.01,0.0,0.313054,0.177071,4928.0,2
002c9a2f,17-AAG,0.131403,0.119089,2.599571e-02,0.097452,NaN,NaN,NaN,NaN,NaN,...,AUC_IC50,KNeighborsRegressor,0.109890,NaN,0.01,0.0,0.140952,0.126641,5798.0,1
012fc42b,17-AAG,0.999991,0.000580,7.094396e-07,0.000419,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
015c61ad,17-AAG,-0.000663,0.264808,9.612650e-02,0.270961,0.5,True,True,0.25,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
02173936,17-AAG,0.999953,0.001263,4.061455e-06,0.000800,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff8b2c6a,17-AAG,0.588417,0.154572,3.874948e-02,0.127621,1.0,True,True,0.00,5000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffa1a16e,17-AAG,0.872258,0.079869,1.063541e-02,0.067162,NaN,NaN,NaN,NaN,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ffb505bc,17-AAG,0.999988,0.000610,9.238533e-07,0.000424,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
lo.groupby('model').mean()

,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,alpha,l1_ratio,max_iter,random_state,tol,max_depth,...,epsilon,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds
model,,,,,,,,,,,,,,,,,,,,,
DecisionTreeRegressor,-0.094432,0.165591,0.049621,0.138739,NaN,NaN,NaN,NaN,NaN,2.518900,...,NaN,4.0,20.0,0.090949,NaN,0.01,0.0,-0.094432,0.338541,3467.963918
ElasticNet,0.109282,0.152390,0.040928,0.135200,2.924088,0.108212,5000.0,NaN,0.0001,NaN,...,NaN,4.0,20.0,0.092379,NaN,0.01,0.0,0.109282,0.121841,2296.259854
KNeighborsRegressor,0.068197,0.152170,0.042126,0.131089,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,20.0,0.094594,NaN,0.01,0.0,0.068197,0.107072,10402.251889
RandomForestRegressor,0.108255,0.161446,0.044138,0.143393,NaN,NaN,NaN,NaN,NaN,4.345299,...,NaN,4.0,20.0,0.079032,NaN,0.01,0.0,0.108255,0.092693,4008.966154
SVR,0.015268,0.160099,0.044577,0.137407,NaN,NaN,-1.0,NaN,0.0010,NaN,...,0.1,4.0,20.0,0.067049,NaN,0.01,0.0,0.015268,0.295948,1913.851003


In [86]:
lo.groupby('alpha').mean()

,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,l1_ratio,max_iter,random_state,tol,max_depth,max_leaf_nodes,...,epsilon,cutoff,drugs,n,test,threshold,step,r_2_mean,r_2_std,seconds
alpha,,,,,,,,,,,,,,,,,,,,,
0.5,0.002543,0.166149,0.047929,0.158051,0.270073,5000.0,NaN,0.0001,NaN,NaN,...,NaN,4.0,20.0,0.103503,NaN,0.01,0.0,0.036990,0.072656,3061.775547
1.0,0.079376,0.143440,0.038705,0.114533,0.010417,5000.0,NaN,0.0001,NaN,NaN,...,NaN,4.0,20.0,0.066855,NaN,0.01,0.0,0.175738,0.158090,1451.125000
1.5,0.243004,0.148359,0.038900,0.124287,0.000000,5000.0,NaN,0.0001,NaN,NaN,...,NaN,4.0,20.0,0.065209,NaN,0.01,0.0,0.168318,0.165310,1704.333333
2.0,0.241096,0.145100,0.036446,0.122128,0.000000,5000.0,NaN,0.0001,NaN,NaN,...,NaN,4.0,20.0,0.070046,NaN,0.01,0.0,0.180594,0.151122,1984.696970
5.0,0.176092,0.142859,0.036071,0.119802,0.000000,5000.0,NaN,0.0001,NaN,NaN,...,NaN,4.0,20.0,0.087637,NaN,0.01,0.0,0.154335,0.154478,1781.500000


In [84]:
k.groupby('normalize').mean()

,r2_score,mean_absolute_error,mean_squared_error,median_absolute_error,alpha,copy_X,fit_intercept,l1_ratio,max_iter,positive,precompute,random_state,tol,warm_start
normalize,,,,,,,,,,,,,,
False,0.300160,0.137149,0.032678,0.114441,0.787859,1.0,1.000000,0.012693,1000,0.0,0.0,NaN,0.0001,0.0
True,0.218647,0.124107,0.028921,0.107119,0.490850,1.0,0.496732,0.043301,1000,0.0,0.0,NaN,0.0001,0.0


In [34]:
plt.scatter(k['l1_ratio'], k['r2_score'])

NameError: name 'k' is not defined